# **Programmation asynchrone avec `asyncio` et `aiohttp`**

## ***Introduction***

La **programmation asynchrone** est un paradigme de programmation qui permet d'exécuter plusieurs tâches de manière **concurrente** sans bloquer l'exécution du programme principal. Au lieu d'attendre qu'une opération se termine avant de passer à la suivante (programmation synchrone), la programmation asynchrone permet de démarrer plusieurs opérations et de les faire progresser simultanément.

**Pourquoi utiliser la programmation asynchrone ?**

En programmation traditionnelle (synchrone), lorsqu'une opération prend du temps (comme une requête réseau, la lecture d'un fichier, ou un appel à une base de données), le programme s'arrête et attend que l'opération se termine. C'est inefficace, surtout quand on doit effectuer plusieurs opérations similaires.

**Exemple concret :**
- Télécharger 10 images depuis Internet
- **Approche synchrone** : télécharger une image, attendre qu'elle soit finie, puis télécharger la suivante → **lent**
- **Approche asynchrone** : démarrer le téléchargement des 10 images en même temps → **rapide**

**Ce que vous allez apprendre :**
- Comprendre les concepts de base de la programmation asynchrone
- Utiliser `asyncio` pour créer des fonctions asynchrones
- Faire des requêtes HTTP asynchrones avec `aiohttp`
- Gérer la concurrence et la synchronisation
- Optimiser les performances avec l'asynchrone
- Utiliser `await` dans les notebooks Jupyter

**⚠️ Note importante pour les notebooks :**
Dans un notebook Jupyter, nous utiliserons directement `await` au lieu de `asyncio.run()` car l'environnement notebook gère déjà la boucle d'événements asynchrone.

# **Partie 1 : Concepts de base de la programmation asynchrone**

## ***Synchrone vs Asynchrone***

### **Programmation synchrone (classique) :**
- Les instructions s'exécutent **une par une**, dans l'ordre
- Chaque instruction **bloque** l'exécution jusqu'à sa completion
- Simple à comprendre et déboguer
- Inefficace pour les opérations d'entrée/sortie (I/O)

### **Programmation asynchrone :**
- Plusieurs opérations peuvent s'exécuter **en parallèle**
- Les opérations longues ne **bloquent pas** le programme
- Plus complexe mais beaucoup plus **efficace**
- Idéale pour les opérations réseau, fichiers, bases de données

### **Concepts clés :**

| Terme | Définition |
|-------|------------|
| **Coroutine** | Fonction spéciale qui peut être suspendue et reprise |
| **`async`** | Mot-clé pour définir une fonction asynchrone |
| **`await`** | Mot-clé pour attendre le résultat d'une opération asynchrone |
| **Event Loop** | Gestionnaire qui orchestre l'exécution des coroutines |
| **Task** | Unité d'exécution d'une coroutine |
| **Future** | Objet qui représente un résultat à venir |

## ***Exemple pratique : Synchrone vs Asynchrone***

Imaginons que nous devons simuler plusieurs tâches qui prennent du temps (comme des appels réseau). Voyons la différence entre les deux approches :

In [ ]:
import time
import asyncio

# === APPROCHE SYNCHRONE ===
print("=== EXEMPLE SYNCHRONE ===")

def tache_longue_sync(nom, duree):
    """Simule une tâche qui prend du temps (version synchrone)"""
    print(f"🔄 Début de la tâche '{nom}' (durée: {duree}s)")
    time.sleep(duree)  # Simule une opération longue
    print(f"✅ Fin de la tâche '{nom}'")
    return f"Résultat de {nom}"

# Mesure du temps d'exécution synchrone
debut = time.time()

# Exécution séquentielle
resultat1 = tache_longue_sync("Téléchargement 1", 2)
resultat2 = tache_longue_sync("Téléchargement 2", 1)
resultat3 = tache_longue_sync("Téléchargement 3", 1.5)

fin = time.time()
print(f"\n⏱️ Temps total (synchrone) : {fin - debut:.2f} secondes")
print(f"📊 Résultats : {[resultat1, resultat2, resultat3]}")

In [ ]:
# === APPROCHE ASYNCHRONE ===
print("\n=== EXEMPLE ASYNCHRONE ===")

async def tache_longue_async(nom, duree):
    """Simule une tâche qui prend du temps (version asynchrone)"""
    print(f"🔄 Début de la tâche '{nom}' (durée: {duree}s)")
    await asyncio.sleep(duree)  # Simule une opération longue SANS bloquer
    print(f"✅ Fin de la tâche '{nom}'")
    return f"Résultat de {nom}"

# Mesure du temps d'exécution asynchrone
debut = time.time()

# Exécution concurrente avec await (spécifique aux notebooks)
resultats = await asyncio.gather(
    tache_longue_async("Téléchargement 1", 2),
    tache_longue_async("Téléchargement 2", 1),
    tache_longue_async("Téléchargement 3", 1.5)
)

fin = time.time()
print(f"\n⏱️ Temps total (asynchrone) : {fin - debut:.2f} secondes")
print(f"📊 Résultats : {resultats}")

### **Analyse des résultats :**

**🐌 Version synchrone :** Environ 4.5 secondes (2 + 1 + 1.5 = somme des durées)
- Les tâches s'exécutent **une après l'autre**
- Chaque `time.sleep()` bloque complètement l'exécution

**⚡ Version asynchrone :** Environ 2 secondes (max(2, 1, 1.5) = durée la plus longue)
- Les tâches s'exécutent **en parallèle**
- `await asyncio.sleep()` permet aux autres tâches de continuer

**Gains de performance :** Jusqu'à **2.25x plus rapide** ! 🚀

### **Points clés à retenir :**

1. **`async def`** : Définit une fonction asynchrone (coroutine)
2. **`await`** : Suspend la fonction actuelle et permet à d'autres de s'exécuter
3. **`asyncio.sleep()`** : Version asynchrone de `time.sleep()`
4. **`asyncio.gather()`** : Exécute plusieurs coroutines en parallèle
5. **Dans les notebooks** : Utilisation directe d'`await` (pas besoin d'`asyncio.run()`)

# **Partie 2 : Syntaxe de base avec `asyncio`**

## ***Créer une fonction asynchrone***

Une fonction asynchrone (coroutine) se définit avec le mot-clé `async` :

In [ ]:
# === FONCTION ASYNCHRONE SIMPLE ===

async def dire_bonjour(nom):
    """Fonction asynchrone simple"""
    await asyncio.sleep(1)  # Simule une opération qui prend du temps
    return f"Bonjour {nom} !"

# Dans un notebook, on peut utiliser directement await
resultat = await dire_bonjour("Alice")
print(resultat)

# === FONCTION AVEC PLUSIEURS AWAIT ===

async def faire_des_calculs():
    """Fonction avec plusieurs opérations asynchrones"""
    print("🔢 Début des calculs...")
    
    # Première opération
    await asyncio.sleep(0.5)
    print("📊 Calcul 1 terminé")
    
    # Deuxième opération
    await asyncio.sleep(0.3)
    print("📈 Calcul 2 terminé")
    
    # Résultat final
    await asyncio.sleep(0.2)
    print("✅ Tous les calculs terminés")
    return "Résultats des calculs"

resultat_calculs = await faire_des_calculs()
print(f"Résultat : {resultat_calculs}")

## ***Exécuter plusieurs tâches simultanément***

### **Méthode 1 : `asyncio.gather()`**
Exécute plusieurs coroutines en parallèle et attend que toutes se terminent :

In [ ]:
# === ASYNCIO.GATHER() ===

async def traiter_commande(commande_id, temps_traitement):
    """Simule le traitement d'une commande"""
    print(f"🛒 Début traitement commande {commande_id}")
    await asyncio.sleep(temps_traitement)
    print(f"✅ Commande {commande_id} traitée")
    return f"Commande_{commande_id}_OK"

# Traiter plusieurs commandes en parallèle
print("=== TRAITEMENT DE COMMANDES EN PARALLÈLE ===")
debut = time.time()

resultats = await asyncio.gather(
    traiter_commande("A001", 1.2),
    traiter_commande("A002", 0.8),
    traiter_commande("A003", 1.5),
    traiter_commande("A004", 0.5)
)

fin = time.time()
print(f"\n⏱️ Temps total : {fin - debut:.2f} secondes")
print(f"📦 Commandes traitées : {resultats}")

# === ASYNCIO.CREATE_TASK() ===
print("\n=== AVEC CREATE_TASK ===")

async def surveiller_serveur(nom_serveur, intervalle):
    """Simule la surveillance d'un serveur"""
    for i in range(3):
        await asyncio.sleep(intervalle)
        print(f"🖥️ {nom_serveur} - Ping {i+1} : OK")
    return f"{nom_serveur} surveillé"

# Créer des tâches
task1 = asyncio.create_task(surveiller_serveur("Serveur-Web", 0.5))
task2 = asyncio.create_task(surveiller_serveur("Serveur-DB", 0.7))
task3 = asyncio.create_task(surveiller_serveur("Serveur-API", 0.3))

# Attendre que toutes les tâches se terminent
resultats_surveillance = await asyncio.gather(task1, task2, task3)
print(f"\n📊 Surveillance terminée : {resultats_surveillance}")

# **Partie 3 : Requêtes HTTP asynchrones avec `aiohttp`**

## ***Introduction à `aiohttp`***

`aiohttp` est une bibliothèque qui permet de faire des requêtes HTTP de manière asynchrone. C'est l'équivalent asynchrone de la bibliothèque `requests` que vous avez vue précédemment.

**Avantages d'`aiohttp` :**
- **Requêtes non-bloquantes** : peut faire plusieurs requêtes simultanément
- **Performance** : idéal pour faire de nombreux appels API
- **Intégration** : fonctionne parfaitement avec `asyncio`

### **Installation :**
```bash
pip install aiohttp
```

**⚠️ Note :** Si `aiohttp` n'est pas installé, certains exemples ne fonctionneront pas. Nous utiliserons des exemples avec `asyncio` et des simulations pour commencer.

In [ ]:
# === SIMULATION DE REQUÊTES HTTP ASYNCHRONES ===

import random

async def simuler_requete_http(url, duree_min=0.5, duree_max=2.0):
    """Simule une requête HTTP avec un temps de réponse aléatoire"""
    duree = random.uniform(duree_min, duree_max)
    
    print(f"Requête vers {url} (temps estimé: {duree:.2f}s)")
    await asyncio.sleep(duree)
    
    # Simule différents codes de statut
    statut = random.choice([200, 200, 200, 404, 500])  # Majorité de succès
    
    if statut == 200:
        print(f"✅ {url} - Succès (200)")
        return {"url": url, "status": 200, "data": f"Données de {url}"}
    else:
        print(f"❌ {url} - Erreur ({statut})")
        return {"url": url, "status": statut, "data": None}

# === COMPARAISON SYNC VS ASYNC POUR LES REQUÊTES ===

urls = [
    "https://api.exemple1.com/users",
    "https://api.exemple2.com/posts", 
    "https://api.exemple3.com/products",
    "https://api.exemple4.com/orders",
    "https://api.exemple5.com/analytics"
]

print("=== SIMULATION REQUÊTES SYNCHRONES ===")
debut_sync = time.time()

resultats_sync = []
for url in urls:
    # En mode synchrone, on attendrait chaque requête
    duree = random.uniform(0.5, 2.0)
    print(f"Requête synchrone vers {url}")
    time.sleep(duree)  # Simule l'attente
    print(f"✅ {url} - Réponse reçue")
    resultats_sync.append({"url": url, "status": 200})

fin_sync = time.time()
print(f"\n⏱️ Temps total (synchrone) : {fin_sync - debut_sync:.2f} secondes")

print("\n=== SIMULATION REQUÊTES ASYNCHRONES ===")
debut_async = time.time()

# Toutes les requêtes en parallèle
resultats_async = await asyncio.gather(*[
    simuler_requete_http(url) for url in urls
])

fin_async = time.time()
print(f"\n⏱️ Temps total (asynchrone) : {fin_async - debut_async:.2f} secondes")
print(f"Accélération : {(fin_sync - debut_sync) / (fin_async - debut_async):.1f}x plus rapide !")

print(f"\n📊 Résultats asynchrones :")
for resultat in resultats_async:
    status_emoji = "✅" if resultat["status"] == 200 else "❌"
    print(f"  {status_emoji} {resultat['url']} - Status: {resultat['status']}")

In [ ]:
# === EXEMPLE RÉEL AVEC AIOHTTP ===

try:
    import aiohttp
    aiohttp_available = True
    print("✅ aiohttp est disponible ! Nous pouvons faire de vraies requêtes.")
except ImportError:
    aiohttp_available = False
    print("❌ aiohttp n'est pas installé. Installez-le avec : pip install aiohttp")

if aiohttp_available:
    async def faire_requete_reelle(url):
        """Fait une vraie requête HTTP avec aiohttp"""
        try:
            async with aiohttp.ClientSession() as session:
                print(f"Requête vers {url}")
                async with session.get(url) as response:
                    data = await response.text()
                    print(f"✅ {url} - Status: {response.status}")
                    return {
                        "url": url,
                        "status": response.status,
                        "taille_reponse": len(data)
                    }
        except Exception as e:
            print(f"❌ Erreur pour {url}: {e}")
            return {"url": url, "status": "error", "error": str(e)}

    # URLs publiques pour tester
    urls_test = [
        "https://httpbin.org/status/200",
        "https://httpbin.org/delay/1",
        "https://httpbin.org/json"
    ]
    
    print("\n=== VRAIES REQUÊTES AIOHTTP ===")
    debut = time.time()
    
    try:
        resultats_reels = await asyncio.gather(*[
            faire_requete_reelle(url) for url in urls_test
        ])
        
        fin = time.time()
        print(f"\n⏱️ Temps total (vraies requêtes) : {fin - debut:.2f} secondes")
        
        for resultat in resultats_reels:
            print(f"{resultat}")
            
    except Exception as e:
        print(f"❌ Erreur lors des requêtes : {e}")
        print("💡 Cela peut arriver si vous n'avez pas d'accès Internet.")
        
else:
    print("\n💡 Pour utiliser aiohttp, installez-le d'abord :")
    print("   pip install aiohttp")
    print("\nVoici la syntaxe de base que vous pourrez utiliser :")
    print("""
# Exemple de syntaxe aiohttp :
async def faire_requete(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            data = await response.json()  # ou .text() pour du texte
            return data
""")

# **Partie 4 : Gestion des erreurs et cas avancés**

## ***Gestion des erreurs dans le code asynchrone***

La gestion des erreurs en programmation asynchrone nécessite une attention particulière, car les erreurs peuvent se produire dans différentes coroutines simultanément.

In [ ]:
# === GESTION D'ERREURS ASYNCHRONES ===

async def operation_qui_peut_echouer(nom, probabilite_echec=0.3):
    """Opération qui peut échouer de manière aléatoire"""
    await asyncio.sleep(random.uniform(0.5, 1.5))
    
    if random.random() < probabilite_echec:
        raise Exception(f"Erreur simulée dans {nom}")
    
    return f"Succès pour {nom}"

# === MÉTHODE 1: TRY/EXCEPT INDIVIDUEL ===
print("=== GESTION D'ERREURS INDIVIDUELLE ===")

async def operation_securisee(nom):
    """Encapsule une opération avec gestion d'erreur"""
    try:
        resultat = await operation_qui_peut_echouer(nom)
        print(f"✅ {resultat}")
        return {"nom": nom, "status": "success", "resultat": resultat}
    except Exception as e:
        print(f"❌ Erreur pour {nom}: {e}")
        return {"nom": nom, "status": "error", "erreur": str(e)}

# Test avec plusieurs opérations
operations = ["Opération-A", "Opération-B", "Opération-C", "Opération-D"]
resultats_securises = await asyncio.gather(*[
    operation_securisee(nom) for nom in operations
])

print(f"\n📊 Résultats sécurisés:")
for resultat in resultats_securises:
    status_emoji = "✅" if resultat["status"] == "success" else "❌"
    print(f"  {status_emoji} {resultat['nom']}: {resultat['status']}")

# === MÉTHODE 2: GESTION D'ERREURS AVEC RETURN_EXCEPTIONS ===
print(f"\n=== AVEC RETURN_EXCEPTIONS ===")

try:
    resultats_avec_exceptions = await asyncio.gather(
        operation_qui_peut_echouer("Test-1"),
        operation_qui_peut_echouer("Test-2"),
        operation_qui_peut_echouer("Test-3"),
        return_exceptions=True  # Ne lève pas d'exception, les retourne
    )
    
    print(f"📊 Résultats avec exceptions:")
    for i, resultat in enumerate(resultats_avec_exceptions):
        if isinstance(resultat, Exception):
            print(f"  ❌ Test-{i+1}: {resultat}")
        else:
            print(f"  ✅ Test-{i+1}: {resultat}")
            
except Exception as e:
    print(f"Erreur globale: {e}")

## ***Timeouts et contrôle de concurrence***

### **Gérer les timeouts avec `asyncio.wait_for()`**

In [ ]:
# === TIMEOUTS ===

async def operation_lente(nom, duree):
    """Opération qui peut prendre beaucoup de temps"""
    print(f"Début de {nom} (durée: {duree}s)")
    await asyncio.sleep(duree)
    print(f"✅ Fin de {nom}")
    return f"Résultat de {nom}"

print("=== GESTION DES TIMEOUTS ===")

# Test avec timeout
async def tester_avec_timeout():
    try:
        # Opération qui doit se terminer en moins de 2 secondes
        resultat = await asyncio.wait_for(
            operation_lente("Opération-Rapide", 1), 
            timeout=2.0
        )
        print(f"✅ Succès: {resultat}")
    except asyncio.TimeoutError:
        print("❌ Timeout: L'opération a pris trop de temps")

await tester_avec_timeout()

# Test d'une opération qui va timeout
async def tester_timeout_echec():
    try:
        # Opération de 3 secondes avec timeout de 1 seconde
        resultat = await asyncio.wait_for(
            operation_lente("Opération-Lente", 3), 
            timeout=1.0
        )
        print(f"✅ Succès: {resultat}")
    except asyncio.TimeoutError:
        print("❌ Timeout: L'opération a pris trop de temps (normal)")

await tester_timeout_echec()

# === CONTRÔLE DE CONCURRENCE AVEC SEMAPHORE ===
print(f"\n=== CONTRÔLE DE CONCURRENCE ===")

# Limiter le nombre d'opérations simultanées
semaphore = asyncio.Semaphore(2)  # Maximum 2 opérations en parallèle

async def operation_avec_limite(nom):
    """Opération limitée par un semaphore"""
    async with semaphore:
        print(f"🔄 {nom} démarre (semaphore acquis)")
        await asyncio.sleep(random.uniform(1, 2))
        print(f"✅ {nom} terminé (semaphore libéré)")
        return f"Résultat de {nom}"

# Lancer 5 opérations, mais seulement 2 en parallèle maximum
debut = time.time()
resultats_limites = await asyncio.gather(*[
    operation_avec_limite(f"Tâche-{i+1}") for i in range(5)
])
fin = time.time()

print(f"\n⏱️ Temps total avec limite de concurrence: {fin - debut:.2f}s")
print("📊 Vous devriez voir que seulement 2 tâches s'exécutent simultanément")

# **Partie 5 : Bonnes pratiques et cas d'usage**

## ***Quand utiliser la programmation asynchrone ?***

### **✅ Utilisez l'asynchrone pour :**
- **Requêtes HTTP/API** : Appels vers des services externes
- **Accès aux bases de données** : Requêtes qui peuvent prendre du temps
- **Opérations sur les fichiers** : Lecture/écriture de gros fichiers
- **WebSockets** : Communication en temps réel
- **Tâches d'attente** : Toute opération qui implique d'attendre

### **❌ N'utilisez PAS l'asynchrone pour :**
- **Calculs intensifs** : Algorithmes qui utilisent beaucoup le CPU
- **Code simple et rapide** : Si tout se fait en mémoire rapidement
- **Traitement d'images/vidéos** : Utilise le CPU, pas d'I/O
- **Opérations mathématiques complexes** : Mieux vaut le multiprocessing

### **🎯 Règle d'or :**
L'asynchrone est parfait pour les opérations **I/O-bound** (limitées par les entrées/sorties), pas pour les opérations **CPU-bound** (limitées par le processeur).

In [ ]:
# === EXEMPLE PRATIQUE COMPLET ===
# Système de surveillance de plusieurs services web

import json

class MoniteurServices:
    """Classe pour surveiller plusieurs services de manière asynchrone"""
    
    def __init__(self):
        self.services = {
            "API-Users": "https://jsonplaceholder.typicode.com/users/1",
            "API-Posts": "https://jsonplaceholder.typicode.com/posts/1", 
            "API-Albums": "https://jsonplaceholder.typicode.com/albums/1"
        }
        self.semaphore = asyncio.Semaphore(3)  # Max 3 requêtes simultanées
    
    async def verifier_service(self, nom, url):
        """Vérifie un service spécifique"""
        async with self.semaphore:
            try:
                print(f"🔍 Vérification de {nom}...")
                
                # Simulation d'une vérification (remplace par vraie requête si aiohttp dispo)
                await asyncio.sleep(random.uniform(0.5, 2.0))
                
                # Simule différents résultats
                if random.random() > 0.2:  # 80% de succès
                    latence = random.uniform(50, 500)
                    print(f"✅ {nom} - OK (latence: {latence:.0f}ms)")
                    return {
                        "service": nom,
                        "status": "OK",
                        "latence_ms": latence,
                        "timestamp": time.time()
                    }
                else:
                    print(f"❌ {nom} - Service indisponible")
                    return {
                        "service": nom,
                        "status": "ERROR",
                        "error": "Service indisponible",
                        "timestamp": time.time()
                    }
                    
            except Exception as e:
                print(f"💥 {nom} - Erreur: {e}")
                return {
                    "service": nom,
                    "status": "EXCEPTION",
                    "error": str(e),
                    "timestamp": time.time()
                }
    
    async def surveiller_tous_services(self, timeout=5.0):
        """Surveille tous les services avec timeout"""
        print("🚀 Début de la surveillance des services...")
        debut = time.time()
        
        try:
            # Surveillance avec timeout global
            resultats = await asyncio.wait_for(
                asyncio.gather(*[
                    self.verifier_service(nom, url) 
                    for nom, url in self.services.items()
                ]),
                timeout=timeout
            )
            
            fin = time.time()
            
            # Analyse des résultats
            services_ok = sum(1 for r in resultats if r["status"] == "OK")
            services_ko = len(resultats) - services_ok
            
            print(f"\n📊 RAPPORT DE SURVEILLANCE")
            print(f"⏱️ Durée totale: {fin - debut:.2f}s")
            print(f"✅ Services OK: {services_ok}")
            print(f"❌ Services KO: {services_ko}")
            print(f"📈 Disponibilité: {services_ok/len(resultats)*100:.1f}%")
            
            return resultats
            
        except asyncio.TimeoutError:
            print(f"⏰ Timeout: La surveillance a pris plus de {timeout}s")
            return []

# === UTILISATION DU MONITEUR ===
print("=== SYSTÈME DE SURVEILLANCE ASYNCHRONE ===")

moniteur = MoniteurServices()
resultats_surveillance = await moniteur.surveiller_tous_services(timeout=10.0)

if resultats_surveillance:
    print(f"\n📋 Détails par service:")
    for resultat in resultats_surveillance:
        status_emoji = {"OK": "✅", "ERROR": "❌", "EXCEPTION": "💥"}
        emoji = status_emoji.get(resultat["status"], "❓")
        
        if resultat["status"] == "OK":
            print(f"  {emoji} {resultat['service']}: {resultat['latence_ms']:.0f}ms")
        else:
            print(f"  {emoji} {resultat['service']}: {resultat.get('error', 'Erreur inconnue')}")

## ***Bonnes pratiques pour l'asynchrone***

### **Conseils essentiels :**

1. **Utilisez `async`/`await` correctement**
   ```python
   # ✅ Correct
   async def ma_fonction():
       resultat = await operation_async()
       return resultat
   
   # ❌ Incorrect (oubli d'await)
   async def ma_fonction():
       resultat = operation_async()  # Retourne une coroutine, pas le résultat !
       return resultat
   ```

2. **Gérez les erreurs**
   - Utilisez `try`/`except` dans les fonctions async
   - Utilisez `return_exceptions=True` avec `gather()` si nécessaire

3. **Contrôlez la concurrence**
   - Utilisez `Semaphore` pour limiter les opérations simultanées
   - N'abusez pas de la parallélisation (plus n'est pas toujours mieux)

4. **Timeouts**
   - Toujours définir des timeouts pour éviter les blocages
   - Utilisez `asyncio.wait_for()` pour les opérations critiques

5. **Dans les notebooks Jupyter**
   - Utilisez directement `await` (pas besoin d'`asyncio.run()`)
   - L'environnement gère la boucle d'événements pour vous

### **🆚 Comparaison des approches de concurrence en Python :**

| Approche | Avantages | Inconvénients | Cas d'usage |
|----------|-----------|---------------|-------------|
| **Threading** | Simple, bon pour I/O | GIL limite CPU, complexité | I/O basique |
| **Multiprocessing** | Vrai parallélisme CPU | Coûteux, communication complexe | Calculs intensifs |
| **Asyncio** | Efficace pour I/O, léger | Courbe d'apprentissage | I/O réseau, APIs |

# **Exercices pratiques**

## ***Exercice 1 : Téléchargeur de fichiers simulé***

Créez une fonction `telecharger_fichier(nom_fichier, taille_mb)` qui :
- Simule le téléchargement d'un fichier (utilisez `asyncio.sleep(taille_mb * 0.1)`)
- Affiche la progression
- Retourne des informations sur le fichier téléchargé

Puis téléchargez 5 fichiers de tailles différentes en parallèle.

In [ ]:
# À vous de jouer ! Exercice 1
# Créez votre fonction telecharger_fichier() ici


## ***Exercice 2 : Gestionnaire de tâches avec priorité***

Créez un système qui :
1. Prend une liste de tâches avec des priorités (1=haute, 2=moyenne, 3=basse)
2. Exécute d'abord toutes les tâches de haute priorité
3. Puis les tâches de moyenne priorité, etc.
4. Utilise un semaphore pour limiter les tâches simultanées

## ***Exercice 3 : API de météo simulée***

Créez une fonction qui simule des appels à différentes stations météo :
- Chaque station a un temps de réponse différent
- Certaines stations peuvent être indisponibles (erreur)
- Retournez un rapport météo combiné avec gestion d'erreurs
- Utilisez un timeout global

In [ ]:
# À vous de jouer ! Exercice 2
# Créez votre gestionnaire de tâches avec priorité ici

In [ ]:
# À vous de jouer ! Exercice 3
# Créez votre API de météo simulée ici

# **Conclusion et aller plus loin**

## ***Récapitulatif des concepts appris***

Félicitations ! Vous maîtrisez maintenant les bases de la programmation asynchrone en Python. Voici ce que vous avez appris :

### **Concepts clés :**
- **Programmation asynchrone vs synchrone** : Comprendre les différences et avantages
- **`async`/`await`** : Syntaxe de base pour créer et utiliser des coroutines
- **`asyncio.gather()`** : Exécuter plusieurs tâches en parallèle
- **Gestion d'erreurs** : `try`/`except` et `return_exceptions=True`
- **Timeouts** : `asyncio.wait_for()` pour éviter les blocages
- **Contrôle de concurrence** : `Semaphore` pour limiter les opérations simultanées
- **`aiohttp`** : Requêtes HTTP asynchrones (optionnel)

### **Points importants à retenir :**

1. **Utilisez l'asynchrone pour les opérations I/O-bound** (réseau, fichiers, BDD)
2. **Dans les notebooks Jupyter, utilisez directement `await`** (pas `asyncio.run()`)
3. **Gérez toujours les erreurs** et définissez des timeouts
4. **Plus de parallélisme n'est pas toujours mieux** - optimisez selon le contexte

## ***Ressources pour aller plus loin***

### **Documentation officielle :**
- [Asyncio Documentation](https://docs.python.org/3/library/asyncio.html)
- [aiohttp Documentation](https://docs.aiohttp.org/)

### **Bibliothèques utiles :**
- **`aiofiles`** : Lecture/écriture de fichiers asynchrone
- **`aiopg`** / **`aiomysql`** : Accès aux bases de données asynchrone
- **`websockets`** : WebSockets asynchrones
- **`httpx`** : Alternative moderne à aiohttp

### **Projets pour pratiquer :**
1. **Crawler web** : Télécharger et analyser plusieurs pages web
2. **Monitoring système** : Surveiller plusieurs services en temps réel
3. **Bot Discord/Telegram** : Créer un bot qui répond à plusieurs utilisateurs
4. **API Gateway** : Agréger plusieurs APIs en une seule

### **La programmation asynchrone est particulièrement utile pour :**
- Applications web (FastAPI, aiohttp)
- Microservices et APIs
- Bots et automatisation
- Traitement de données en temps réel
- IoT et systèmes distribués

**Réalisé par [Benjamin QUINET](https://www.linkedin.com/in/benjamin-quinet-freelance-dev-data-ia)**